In [1]:
import numpy as np, pandas as pd
from scipy.sparse import coo_matrix
from implicit.bpr import BayesianPersonalizedRanking
import recometrics
from poismf import PoisMF
from pandas.api.types import CategoricalDtype
from scipy import sparse
from sklearn.model_selection import train_test_split
import recmetrics
import tqdm


In [ ]:
data = pd.read_csv("ml100k_ratings.csv",sep=",")


In [ ]:
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)


In [4]:
user_positives = train_df.groupby('userId')['movieId'].apply(set).to_dict()


In [ ]:
# from https://www.geeksforgeeks.org/python-pandas-dataframe-where/
users = train_df['userId'].unique()
items = train_df['movieId'].unique()
users = sorted(users)
items = sorted(items)

# HW code
ratingMatrix = test_df.pivot(index='userId', columns='movieId', values='rating')
ratingMatrix = ratingMatrix.reindex(index=users, columns=items)
ratingMatrix.fillna(0)

display(ratingMatrix)

movieId,1,2,3,4,5,6,7,8,9,10,...,1668,1670,1671,1672,1673,1676,1678,1679,1680,1681
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,3.0,NaN,5.0,NaN,1.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# From https://hippocampus-garden.com/pandas_sparse/
users = train_df["userId"].unique()
movies = train_df["movieId"].unique()
shape = (len(users), len(movies))

# Create indices for users and movies
user_cat = CategoricalDtype(categories=sorted(users), ordered=True)
movie_cat = CategoricalDtype(categories=sorted(movies), ordered=True)
user_index = train_df["userId"].astype(user_cat).cat.codes
movie_index = train_df["movieId"].astype(movie_cat).cat.codes

# Conversion via COO matrix
coo = sparse.coo_matrix((train_df["rating"], (user_index, movie_index)), shape=shape)
csr = coo.tocsr()


In [ ]:
# From https://hippocampus-garden.com/pandas_sparse/
bpr = BayesianPersonalizedRanking(factors=19, learning_rate=0.001,
                                  regularization=0.001, dtype=np.float64,
                                  iterations=100)

In [ ]:
# From https://hippocampus-garden.com/pandas_sparse/
bpr.fit(csr)


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# compute dot product for results
pred = np.matmul(bpr.user_factors,bpr.item_factors.T)

In [10]:
from recsys_metrics import *
import torch
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation

In [ ]:
ratingMatrix[ratingMatrix >1] = 1
true = torch.tensor(ratingMatrix.fillna(0).to_numpy())
# true result needs to be be binary values for metric evaluation

In [ ]:
predv = torch.tensor(pred)
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation

In [21]:
display(true)
display(predv)

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

tensor([[ 0.5769,  0.0662, -0.0248,  ...,  0.0061, -0.0205, -0.0297],
        [ 0.5893,  0.0612, -0.0271,  ...,  0.0055, -0.0221, -0.0307],
        [ 0.5850,  0.0660, -0.0271,  ...,  0.0049, -0.0230, -0.0322],
        ...,
        [ 0.5863,  0.0644, -0.0283,  ...,  0.0068, -0.0206, -0.0314],
        [ 0.5822,  0.0680, -0.0266,  ...,  0.0068, -0.0213, -0.0301],
        [ 0.5805,  0.0666, -0.0267,  ...,  0.0063, -0.0183, -0.0298]],
       dtype=torch.float64)

In [ ]:
# jinming code
def calculate_metrics(pred_df, user_postivies, k_list = [10,50,100]):
  metrics = {
    'MAR@ALL': [],
    'MAR@1000': [],
    'MRR': [],
  }


  grouped = pred_df.groupby('user')

  for user, user_data in tqdm.tqdm(grouped, desc="Processing Users"):
    positives = user_postivies.get(user, set())
    if len(positives) == 0:
      continue

    sorted_data = user_data.sort_values(by='score', ascending=False)
    all_items = sorted_data['item'].tolist()

    relevant_ranks1k = []
    relevant_ranks_all=[]
    precisions = []
    hit_at_ranks = None
    for rank, item in enumerate(all_items, start=1):
      if item in positives:
        relevant_ranks1k.append(rank)
    #calculate relevant
    for rank, item in enumerate(all_items, start=1):
      if item in positives:
        relevant_ranks_all.append(rank)
        precisions.append(len(positives) / rank)

        if hit_at_ranks is None:
            hit_at_ranks = rank


    rr = 1.0 / hit_at_ranks if hit_at_ranks is not None else 0.0
    metrics['MRR'].append(rr)

    if relevant_ranks_all:
      metrics['MAR@ALL'].append(np.mean(relevant_ranks_all))
    if relevant_ranks1k:
      metrics['MAR@1000'].append(np.mean(relevant_ranks_all))
      
    all_items = sorted_data['item'].tolist()[1000:]
  
  final = {
    'MAR@ALL': np.mean(metrics['MAR@ALL']),
    'MAR@1000': np.mean(metrics['MAR@1000']),
    'MRR': np.mean(metrics['MRR']),
  }

  return final

In [ ]:
# Toss https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation
# functions here as a group
def metricStuff(pred,true,user_positive):
    print("NDCG@10",normalized_dcg(pred, true ,k=10))
    print("MRR@1000",mean_reciprocal_rank(pred, true,k=1000))
    print("MRR@all",mean_reciprocal_rank(pred, true))
    # From https://www.geeksforgeeks.org/how-to-convert-torch-tensor-to-pandas-dataframe/
    marData = pred.detach().numpy()
    marDF = pd.DataFrame(marData)
    marDF.index.name = "user"
    marDF.columns.name = "items"
    # From https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html
    # From https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
    marDF = marDF.stack().reset_index()
    marDF.columns = ['user', 'item', 'score']
    print("MAR",calculate_metrics(marDF,user_positive))
    print("MAP",mean_average_precision(pred, true))
    print("P@50",precision(pred, true,k=50))
    print("R@50",recall(pred, true,k=50))

In [26]:
metricStuff(predv,true,user_positives)

NDCG@10 tensor(0.1125)
MRR@1000 tensor(0.2689)
MRR@all tensor(0.2689)


Processing Users: 100%|██████████| 943/943 [00:00<00:00, 1754.84it/s]

MAR {'MAR@ALL': np.float64(569.6857358559441), 'MAR@1000': np.float64(569.6857358559441), 'MRR': np.float64(0.23975466255741684)}
MAP tensor(0.0774)
P@50 tensor(0.0726)
R@50 tensor(0.1991)
